# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [3]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [4]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [5]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [6]:
np.linalg.norm(BB)


0.36436161983015336

### Do the Jacobi iteration

In [7]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [8]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

array([[15.19151945,  0.62210877,  0.43772774,  0.78535858],
       [ 0.35781727, 15.50099513,  0.68346294,  0.71270203],
       [ 0.36488598,  0.61539618, 15.07538124,  0.36882401],
       [ 0.86912739,  0.43617342,  0.80214764, 15.14376682]])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [17]:
from sklearn.metrics import mean_absolute_error


def Jacobi_iterate(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """
    # 1. Matrix splitting
    D, U, L = np.diag(np.diag(A)), np.triu(A, 1), np.tril(A, -1)

    # 2. Building an inverse matrix
    D_inv = np.linalg.inv(D)

    # 3. Defining objective function
    objective = lambda A, x, b: mean_absolute_error((A @ x), b)
    
    # 4. Initialization of starting x
    x_prev = np.zeros(shape=(A.shape[0]))
    x = np.random.uniform(size=(A.shape[0]))

    # 5. Computing optimization by precomputed components
    x_multiplier = D_inv @ (-L - U)
    offset = D_inv @ b
    
    # 6. Start iteration = 0
    n_iter = 0
    while objective(A, x, b) > eps and n_iter < n_max_iters:
        x_prev = x
        x = x_multiplier @ x + offset
        n_iter += 1
        print(f"[{n_iter}]: {objective(A, x, b)}")
        
    return n_iter, x

_, x = Jacobi_iterate(A, b, n_max_iters = 2000)

[1]: 2.5295469399169472
[2]: 0.7624316098462989
[3]: 0.2283263560168008
[4]: 0.06849511976086387
[5]: 0.02054202498428437
[6]: 0.006160531196011946
[7]: 0.0018475884619915677
[8]: 0.00055409958378996
[9]: 0.0001661772360649752
[10]: 4.983736010933833e-05
[11]: 1.4946467272435731e-05
[12]: 4.482518384572864e-06
[13]: 1.3443291112791994e-06
[14]: 4.031708527273137e-07
[15]: 1.2091290375292053e-07
[16]: 3.6262369206008226e-08
[17]: 1.087526124640692e-08
[18]: 3.261543895724528e-09
[19]: 9.781528722413447e-10
[20]: 2.933528267523577e-10
[21]: 8.79779003121195e-11
[22]: 2.6384996965056472e-11
[23]: 7.912973054580164e-12
[24]: 2.3731402259974387e-12
[25]: 7.116859185307689e-13
[26]: 2.1345009093565183e-13
[27]: 6.402309238318083e-14
[28]: 1.9215531943395092e-14
[29]: 5.7336080327985425e-15
[30]: 1.7340295865864164e-15
[31]: 4.947431353485854e-16


In [18]:
# Verficiation of convergence to actual x
np.allclose(A @ x, b)

True

In [46]:
## NOW lets see, what happens if we reduce our diagonal
_, x = Jacobi_iterate((A - np.diag([10] * n)), b, n_max_iters = 2000)

[1]: 3.342780360390955
[2]: 2.8598856654441613
[3]: 2.4299131322230805
[4]: 2.064899193389599
[5]: 1.7554967754199513
[6]: 1.4922152368425288
[7]: 1.2684616783509686
[8]: 1.078254939595684
[9]: 0.9165698873412523
[10]: 0.7791296829617773
[11]: 0.6622987157852341
[12]: 0.5629866186692928
[13]: 0.47856643290400847
[14]: 0.4068051049470158
[15]: 0.34580443185082194
[16]: 0.29395084681517425
[17]: 0.2498727384152236
[18]: 0.21240416919956173
[19]: 0.1805540347437296
[20]: 0.1534798473358256
[21]: 0.13046545080902155
[22]: 0.1109020770496169
[23]: 0.09427224309310164
[24]: 0.08013606286047052
[25]: 0.06811961145801146
[26]: 0.05790503425742175
[27]: 0.04922213912538166
[28]: 0.041841249403421134
[29]: 0.03556712858780531
[30]: 0.030233816007369026
[31]: 0.02570023689460423
[32]: 0.021846470729258487
[33]: 0.018570579146083736
[34]: 0.01578590949974763
[35]: 0.013418802761828493
[36]: 0.011406645120050522
[37]: 0.009696211741400075
[38]: 0.00824225888897007
[39]: 0.007006327151741579
[40]: 0

<b>Summary</b>: Jacobi iteration applicable only to diagonal dominant matrix, otherwise algorithm will not converge.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [61]:
def Gauss_Seidel_iterate(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """
    # 1. Matrix splitting
    U, LD = np.triu(A, 1), np.tril(A)
    LD_inv = np.linalg.inv(LD)
    print(f"Norm: {np.linalg.norm(LD_inv)}")

    # 2. Defining objective function
    objective = lambda A, x, b: mean_absolute_error((A @ x), b)

    # 3. Initialization of starting x
    x = np.random.uniform(size=(A.shape[0]))

    # 4. definition of converging function
    converge = lambda LD_inv, x, U, b: LD_inv @ (-U @ x + b)

    # 5. Start iteration = 0
    n_iter = 0
    while objective(A, x, b) > eps and n_iter < n_max_iters:
        x = converge(LD_inv, x, U, b)
        n_iter += 1
        print(f"[{n_iter}]: {objective(A, x, b)}")
        
    return n_iter, x

_, x = Gauss_Seidel_iterate(A, b, n_max_iters = 2000)
np.allclose(A @ x, b)

Norm: 0.20499614567534405
[1]: 1.2034907780660622
[2]: 0.06814922223813993
[3]: 0.002851517272069469
[4]: 0.00025002605413072256
[5]: 1.2609407855605423e-05
[6]: 8.03107174433132e-07
[7]: 5.511935533657197e-08
[8]: 2.509381916149245e-09
[9]: 2.1809791542204416e-10
[10]: 1.0196343075419278e-11
[11]: 7.492211712145646e-13
[12]: 4.3556130924216066e-14
[13]: 2.212813265956015e-15
[14]: 2.1163626406917047e-16


True

In [62]:
## Lets research how reduction of diagonal dominance influence on convergence speed
_, x = Gauss_Seidel_iterate((A - np.diag([10] * n)), b, n_max_iters = 2000)

Norm: 0.5898638843769459
[1]: 0.6774729556474421
[2]: 0.1014302929629789
[3]: 0.022897506211702796
[4]: 0.004016424343584913
[5]: 0.0009045993288244177
[6]: 0.00016541119536011908
[7]: 3.440216160104162e-05
[8]: 6.644785628476951e-06
[9]: 1.315237344701875e-06
[10]: 2.7936234166907323e-07
[11]: 5.156227878728992e-08
[12]: 1.16162786938101e-08
[13]: 2.026854137998635e-09
[14]: 4.659159585918005e-10
[15]: 7.778060695518008e-11
[16]: 1.8045080707462803e-11
[17]: 3.179850827095265e-12
[18]: 6.982425054813391e-13
[19]: 1.2548295735825832e-13
[20]: 2.847652669224487e-14
[21]: 5.0594944900339554e-15
[22]: 1.1313866510320737e-15
[23]: 2.563921297493721e-16


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [94]:
def minimum_residual_convergence(A: np.array, b: np.array, eps=1e-15, n_max_iters=1000):
    """
        Arguments:
            A (np.array): initial A matrix
            b (np.array): initial array
            eps (float): minimal error value
            n_max_iter (int): maximum amount of iterations need to be done
            
        Outputs:
            n_iter (int): amount of iterations done
            x (np.array): vector
    """

    # 1. Functions definition & initial computings
    f_residuals = lambda A, x, b: A @ x - b
    f_tao = lambda A, r: (A @ residuals) / np.linalg.norm(A @ residuals)
    objective = lambda residuals: np.linalg.norm(residuals)

    # 2. Initialization of starting x
    x = np.random.uniform(size=(A.shape[0]))         
    residuals = f_residuals(A, x, b)
    
    # 5. Start iteration = 0
    n_iter = 0
    while objective(residuals) > eps and n_iter < n_max_iters:

        # 1. Offsetting components
        residuals = f_residuals(A, x, b)
        tao = f_tao(A, residuals)

        # 2. Computing
        x = x + np.multiply(residuals, tao)
        n_iter += 1
        print(f"[{n_iter}]: {objective(residuals)}")
        
    return n_iter, x

In [95]:
minimum_residual_convergence(A, b)

[1]: 32.32858496286622
[2]: 255.29087812976616
[3]: 2175.7485960601794
[4]: 21472.351043880204
[5]: 258199.09940311546
[6]: 3647194.297064442
[7]: 55816816.673899196
[8]: 877943483.5686679
[9]: 13912098338.271044
[10]: 220863417551.03098
[11]: 3507950256720.858
[12]: 55722681982330.586
[13]: 885161830341738.2
[14]: 1.4061003922412272e+16
[15]: 2.2336273462222672e+17
[16]: 3.548177102343753e+18
[17]: 5.6363753231342715e+19
[18]: 8.953534922664494e+20
[19]: 1.4222932917594135e+22
[20]: 2.2593514499298177e+23
[21]: 3.5890410268062796e+24
[22]: 5.701288966677201e+25
[23]: 9.056652080471807e+26
[24]: 1.4386737347747143e+28
[25]: 2.2853722288797354e+29
[26]: 3.63037574002452e+30
[27]: 5.766950279351323e+31
[28]: 9.160956855745846e+32
[29]: 1.455242831090908e+34
[30]: 2.311692687552853e+35
[31]: 3.672186502151889e+36
[32]: 5.833367808444137e+37
[33]: 9.266462901231179e+38
[34]: 1.4720027524339513e+40
[35]: 2.3383162769531408e+41
[36]: 3.714478795656553e+42
[37]: 5.900550263183518e+43
[38]: 9.

(1000,
 array([7.53536432e+151, 1.09868371e+149, 1.12412433e+149, 6.48460054e+149,
        5.43990600e+147, 1.57532466e+148, 6.99581789e+148, 4.41556804e+149,
        1.70330372e+149, 2.72609786e+148]))